In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import optimizers, layers
from keras.models import Sequential
from keras.layers import Dense, Dropout
import os
import glob
from glob import glob
from tensorflow.keras.preprocessing.text import text_to_word_sequence
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping,ModelCheckpoint
import cv2
import random
from matplotlib import pyplot as plt
from keras.models import Sequential
from sklearn.manifold import TSNE
from sklearn.datasets import load_digits
import pandas as pd
from sklearn.decomposition import PCA

data_dir = os.chdir(r'C:\Users\admin\OneDrive\바탕 화면\code\dataset\image\sample_cw')
data_list = glob('*.jpg')

random.shuffle(data_list)
print(data_list[0:5])
print(text_to_word_sequence(data_list[0]))

FileNotFoundError: [WinError 2] 지정된 파일을 찾을 수 없습니다: 'C:\\Users\\admin\\OneDrive\\바탕 화면\\code\\dataset\\image\\sample_cw'

In [ ]:
label = []
data_height =  32
data_width =32
channel_n = 3
batch_size = len(data_list)
images = np.zeros((batch_size, data_height, data_width, channel_n))
for n, path in enumerate(data_list[:batch_size]):
# lable 
    token = text_to_word_sequence(data_list[n])
    label.append(token[0])
# image transform
    image = cv2.imread(data_list[n])
    image = cv2.resize(image, (32, 32))/255
    images[n, :, :, :] =image
label = np.array(label)

items = label
encoder = LabelEncoder()
encoder.fit(items)
label_encoded = keras.utils.to_categorical(encoder.transform(items))
label = label = encoder.transform(items)

X =images
y = label_encoded

In [ ]:
X_train = X[0:20000]
y_train = y[0:20000]

X_val = X[20000:24000]
y_val = y[20000:24000]

X_test = X[24000:]
y_test = y[24000:]

print(X_train.shape, y_train.shape)
print(X_val.shape, y_val.shape)
print(X_test.shape, y_test.shape)

In [ ]:
model = Sequential()
model.add(layers.Conv2D(128, (3, 3), activation='relu', input_shape=(data_height, data_width, channel_n)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Flatten())
model.add(layers.Dense(32, activation='relu'))
model.add(Dropout(0.25))
model.add(layers.Dense(2, activation='softmax'))

model.summary()

In [ ]:
adam = keras.optimizers.Adam(lr=0.001)
es = [EarlyStopping(monitor ='val_loss',mode='min',patience =20), ModelCheckpoint(filepath='best_model.h5',monitor='val_loss',save_best_only=True)]

model.compile(optimizer=adam,loss='categorical_crossentropy',metrics=['accuracy'])
hist = model.fit(X_train, y_train, epochs=20, batch_size=128, validation_data=(X_val, y_val),callbacks=[es])

In [ ]:
plt.figure(figsize=(12,4))
plt.subplot(1, 2, 1)
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('model accuracy')
plt.xlabel('epoch')
plt.ylabel('accuracy')
plt.legend(['train', 'validation'])

plt.subplot(1, 2, 2)
plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('model loss')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.legend(['train', 'validation'])

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix

loss_and_metrics = model.evaluate(X_test, y_test, batch_size=128)
print('loss_and_metrics : ' + str(loss_and_metrics))

y_pred = model.predict(X_test)
y_pred = np.ceil(y_pred-0.5)
print('y_test:' + str(y_test))
print('y_pred:' + str(y_pred))

cf_matrix = confusion_matrix(y_test, y_pred)
print('cf_matrix:' + str(cf_matrix))
sns.heatmap(cf_matrix/np.sum(cf_matrix), annot=True, fmt = '.2%', cmap='Blues')

In [ ]:
from tensorflow.keras import backend as K
layerIndex = 9
func = K.function([model.get_layer(index=0).input], model.get_layer(index=layerIndex).output)
layerOutput = func([X])  # input_data is a numpy array
print(layerOutput)

pca = PCA(n_components=2)
pca_result = pca.fit_transform(layerOutput)
print(pca_result)

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
pcax_train,pcax_test,pcay_train,pcay_test = train_test_split(pca_result,label,test_size=0.3,random_state=1004)

classifier = KNeighborsClassifier(n_neighbors=11)
classifier.fit(pcax_train, pcay_train)

pca_pred = classifier.predict(pcax_test)

print(confusion_matrix(pcay_test, pca_pred))
print(classification_report(pcay_test, pca_pred))